In [1]:
from cosmikyu import config, gan
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.autograd import Variable
import numpy as np
from torchvision.utils import save_image
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
data_dir = config.default_data_dir
mnist_dir = os.path.join(data_dir, 'mnist')
nepochs = 200
batch_size = 64
cuda = False
learning_rate = 0.00005
latent_dim = 100
shape = (1,28,28)
clip_tresh = 0.01
num_critic = 5
sample_interval = 1000

In [4]:
# Configure data loader
os.makedirs(data_dir, exist_ok=True)
os.makedirs(mnist_dir, exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        mnist_dir,
        train=True,
        download=True,
        transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]),
    ),
    batch_size=batch_size,
    shuffle=True,
)



In [5]:
# Initialize GAN
generator = gan.WGAN_Generator(shape, latent_dim)
discriminator = gan.WGAN_Discriminator(shape)

if cuda:
    generator.cuda()
    discriminator.cuda()
else: pass

# Initialize Optimizers
opt_gen = torch.optim.RMSprop(generator.parameters(), lr=learning_rate)
opt_disc = torch.optim.RMSprop(discriminator.parameters(), lr=learning_rate)

In [6]:
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [7]:
# Traning Time
batches_done = 0
for epoch in range(nepochs):
    
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        opt_disc.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z).detach()
        # Adversarial loss
        loss_D = -torch.mean(discriminator(real_imgs)) + torch.mean(discriminator(fake_imgs))

        loss_D.backward()
        opt_disc.step()

        # Clip weights of discriminator
        for p in discriminator.parameters():
            p.data.clamp_(-clip_tresh, clip_tresh)

        # Train the generator every n_critic iterations
        if i % num_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            opt_gen.zero_grad()

            # Generate a batch of images
            gen_imgs = generator(z)
            # Adversarial loss
            loss_G = -torch.mean(discriminator(gen_imgs))

            loss_G.backward()
            opt_gen.step()

            print(
                "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
                % (epoch, nepochs, batches_done % len(dataloader), len(dataloader), loss_D.item(), loss_G.item())
            )

        if batches_done % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)
        batches_done += 1

[Epoch 0/200] [Batch 0/938] [D loss: -0.093752] [G loss: -0.010411]
[Epoch 0/200] [Batch 5/938] [D loss: -0.112455] [G loss: -0.011327]
[Epoch 0/200] [Batch 10/938] [D loss: -0.350778] [G loss: -0.014561]
[Epoch 0/200] [Batch 15/938] [D loss: -0.742031] [G loss: -0.025720]
[Epoch 0/200] [Batch 20/938] [D loss: -1.206115] [G loss: -0.051232]
[Epoch 0/200] [Batch 25/938] [D loss: -1.741575] [G loss: -0.087430]
[Epoch 0/200] [Batch 30/938] [D loss: -2.385663] [G loss: -0.142975]
[Epoch 0/200] [Batch 35/938] [D loss: -3.000030] [G loss: -0.219349]
[Epoch 0/200] [Batch 40/938] [D loss: -3.606102] [G loss: -0.311039]
[Epoch 0/200] [Batch 45/938] [D loss: -4.252279] [G loss: -0.434292]
[Epoch 0/200] [Batch 50/938] [D loss: -4.887240] [G loss: -0.565729]
[Epoch 0/200] [Batch 55/938] [D loss: -5.342968] [G loss: -0.749850]
[Epoch 0/200] [Batch 60/938] [D loss: -5.980068] [G loss: -0.920219]
[Epoch 0/200] [Batch 65/938] [D loss: -6.560141] [G loss: -1.091699]
[Epoch 0/200] [Batch 70/938] [D loss

[Epoch 0/200] [Batch 590/938] [D loss: -1.403973] [G loss: -18.154652]
[Epoch 0/200] [Batch 595/938] [D loss: -1.278948] [G loss: -17.917641]
[Epoch 0/200] [Batch 600/938] [D loss: -1.755726] [G loss: -17.953552]
[Epoch 0/200] [Batch 605/938] [D loss: -1.445301] [G loss: -17.832020]
[Epoch 0/200] [Batch 610/938] [D loss: -1.840233] [G loss: -17.868366]
[Epoch 0/200] [Batch 615/938] [D loss: -1.349026] [G loss: -17.809040]
[Epoch 0/200] [Batch 620/938] [D loss: -1.070793] [G loss: -17.724457]
[Epoch 0/200] [Batch 625/938] [D loss: -1.390177] [G loss: -17.667120]
[Epoch 0/200] [Batch 630/938] [D loss: -1.879465] [G loss: -17.227482]
[Epoch 0/200] [Batch 635/938] [D loss: -1.296644] [G loss: -17.346418]
[Epoch 0/200] [Batch 640/938] [D loss: -1.353584] [G loss: -17.520113]
[Epoch 0/200] [Batch 645/938] [D loss: -1.689228] [G loss: -17.417183]
[Epoch 0/200] [Batch 650/938] [D loss: -1.145794] [G loss: -17.408342]
[Epoch 0/200] [Batch 655/938] [D loss: -1.064369] [G loss: -17.323109]
[Epoch

[Epoch 1/200] [Batch 230/938] [D loss: -0.446727] [G loss: -14.147347]
[Epoch 1/200] [Batch 235/938] [D loss: -0.436001] [G loss: -14.067598]
[Epoch 1/200] [Batch 240/938] [D loss: -0.480880] [G loss: -14.042624]
[Epoch 1/200] [Batch 245/938] [D loss: -0.401678] [G loss: -14.006895]
[Epoch 1/200] [Batch 250/938] [D loss: -0.475464] [G loss: -13.943729]
[Epoch 1/200] [Batch 255/938] [D loss: -0.335284] [G loss: -13.939473]
[Epoch 1/200] [Batch 260/938] [D loss: -0.478020] [G loss: -13.882062]
[Epoch 1/200] [Batch 265/938] [D loss: -0.542119] [G loss: -13.846544]
[Epoch 1/200] [Batch 270/938] [D loss: -0.370181] [G loss: -13.823237]
[Epoch 1/200] [Batch 275/938] [D loss: -0.465585] [G loss: -13.785255]
[Epoch 1/200] [Batch 280/938] [D loss: -0.382888] [G loss: -13.837615]
[Epoch 1/200] [Batch 285/938] [D loss: -0.454199] [G loss: -13.803788]
[Epoch 1/200] [Batch 290/938] [D loss: -0.387163] [G loss: -13.784700]
[Epoch 1/200] [Batch 295/938] [D loss: -0.396119] [G loss: -13.701501]
[Epoch

[Epoch 1/200] [Batch 810/938] [D loss: -0.168050] [G loss: -12.710952]
[Epoch 1/200] [Batch 815/938] [D loss: -0.183335] [G loss: -12.665854]
[Epoch 1/200] [Batch 820/938] [D loss: -0.141339] [G loss: -12.715263]
[Epoch 1/200] [Batch 825/938] [D loss: -0.202863] [G loss: -12.733810]
[Epoch 1/200] [Batch 830/938] [D loss: -0.199007] [G loss: -12.769833]
[Epoch 1/200] [Batch 835/938] [D loss: -0.215805] [G loss: -12.781908]
[Epoch 1/200] [Batch 840/938] [D loss: -0.100883] [G loss: -12.813332]
[Epoch 1/200] [Batch 845/938] [D loss: -0.059393] [G loss: -12.859180]
[Epoch 1/200] [Batch 850/938] [D loss: -0.118073] [G loss: -12.852473]
[Epoch 1/200] [Batch 855/938] [D loss: -0.069919] [G loss: -12.857067]
[Epoch 1/200] [Batch 860/938] [D loss: -0.075950] [G loss: -12.876959]
[Epoch 1/200] [Batch 865/938] [D loss: -0.088398] [G loss: -12.865169]
[Epoch 1/200] [Batch 870/938] [D loss: -0.102687] [G loss: -12.882727]
[Epoch 1/200] [Batch 875/938] [D loss: -0.133803] [G loss: -12.903893]
[Epoch

[Epoch 2/200] [Batch 455/938] [D loss: -0.017281] [G loss: -11.534153]
[Epoch 2/200] [Batch 460/938] [D loss: -0.003850] [G loss: -11.637312]
[Epoch 2/200] [Batch 465/938] [D loss: -0.013519] [G loss: -11.612452]
[Epoch 2/200] [Batch 470/938] [D loss: -0.057121] [G loss: -11.531871]
[Epoch 2/200] [Batch 475/938] [D loss: -0.025120] [G loss: -11.476769]
[Epoch 2/200] [Batch 480/938] [D loss: -0.027771] [G loss: -11.445717]
[Epoch 2/200] [Batch 485/938] [D loss: -0.018885] [G loss: -11.464376]
[Epoch 2/200] [Batch 490/938] [D loss: 0.008489] [G loss: -11.446138]
[Epoch 2/200] [Batch 495/938] [D loss: -0.054312] [G loss: -11.552301]
[Epoch 2/200] [Batch 500/938] [D loss: -0.048822] [G loss: -11.497358]
[Epoch 2/200] [Batch 505/938] [D loss: 0.003969] [G loss: -11.543149]
[Epoch 2/200] [Batch 510/938] [D loss: -0.030758] [G loss: -11.535103]
[Epoch 2/200] [Batch 515/938] [D loss: -0.056686] [G loss: -11.527386]
[Epoch 2/200] [Batch 520/938] [D loss: -0.019054] [G loss: -11.592076]
[Epoch 2

...

[Epoch 31/200] [Batch 55/938] [D loss: -0.302628] [G loss: -0.556052]
[Epoch 31/200] [Batch 60/938] [D loss: -0.284424] [G loss: -0.576663]
[Epoch 31/200] [Batch 65/938] [D loss: -0.273071] [G loss: -0.649562]
[Epoch 31/200] [Batch 70/938] [D loss: -0.330273] [G loss: -0.659837]
[Epoch 31/200] [Batch 75/938] [D loss: -0.268028] [G loss: -0.724924]
[Epoch 31/200] [Batch 80/938] [D loss: -0.270746] [G loss: -0.657028]
[Epoch 31/200] [Batch 85/938] [D loss: -0.281074] [G loss: -0.673341]
[Epoch 31/200] [Batch 90/938] [D loss: -0.286947] [G loss: -0.709850]
[Epoch 31/200] [Batch 95/938] [D loss: -0.288760] [G loss: -0.777616]
[Epoch 31/200] [Batch 100/938] [D loss: -0.322541] [G loss: -0.792289]
[Epoch 31/200] [Batch 105/938] [D loss: -0.302620] [G loss: -0.780026]
[Epoch 31/200] [Batch 110/938] [D loss: -0.279322] [G loss: -0.788540]
[Epoch 31/200] [Batch 115/938] [D loss: -0.304955] [G loss: -0.697995]
[Epoch 31/200] [Batch 120/938] [D loss: -0.297728] [G loss: -0.697281]
[Epoch 31/200] 

[Epoch 31/200] [Batch 640/938] [D loss: -0.300120] [G loss: -0.451862]
[Epoch 31/200] [Batch 645/938] [D loss: -0.303157] [G loss: -0.407531]
[Epoch 31/200] [Batch 650/938] [D loss: -0.268804] [G loss: -0.426064]
[Epoch 31/200] [Batch 655/938] [D loss: -0.251326] [G loss: -0.512737]
[Epoch 31/200] [Batch 660/938] [D loss: -0.282339] [G loss: -0.529099]
[Epoch 31/200] [Batch 665/938] [D loss: -0.309826] [G loss: -0.507337]
[Epoch 31/200] [Batch 670/938] [D loss: -0.255531] [G loss: -0.528049]
[Epoch 31/200] [Batch 675/938] [D loss: -0.274132] [G loss: -0.598358]
[Epoch 31/200] [Batch 680/938] [D loss: -0.267230] [G loss: -0.630743]
[Epoch 31/200] [Batch 685/938] [D loss: -0.306609] [G loss: -0.698977]
[Epoch 31/200] [Batch 690/938] [D loss: -0.296322] [G loss: -0.706562]
[Epoch 31/200] [Batch 695/938] [D loss: -0.303142] [G loss: -0.676964]
[Epoch 31/200] [Batch 700/938] [D loss: -0.287066] [G loss: -0.751207]
[Epoch 31/200] [Batch 705/938] [D loss: -0.295703] [G loss: -0.822775]
[Epoch

[Epoch 32/200] [Batch 280/938] [D loss: -0.316125] [G loss: -0.472543]
[Epoch 32/200] [Batch 285/938] [D loss: -0.246935] [G loss: -0.514384]
[Epoch 32/200] [Batch 290/938] [D loss: -0.314269] [G loss: -0.434623]
[Epoch 32/200] [Batch 295/938] [D loss: -0.273673] [G loss: -0.442298]
[Epoch 32/200] [Batch 300/938] [D loss: -0.320392] [G loss: -0.346112]
[Epoch 32/200] [Batch 305/938] [D loss: -0.286633] [G loss: -0.355485]
[Epoch 32/200] [Batch 310/938] [D loss: -0.312956] [G loss: -0.303809]
[Epoch 32/200] [Batch 315/938] [D loss: -0.361058] [G loss: -0.177469]
[Epoch 32/200] [Batch 320/938] [D loss: -0.294676] [G loss: -0.177236]
[Epoch 32/200] [Batch 325/938] [D loss: -0.280860] [G loss: -0.167403]
[Epoch 32/200] [Batch 330/938] [D loss: -0.327575] [G loss: -0.178035]
[Epoch 32/200] [Batch 335/938] [D loss: -0.312323] [G loss: -0.178885]
[Epoch 32/200] [Batch 340/938] [D loss: -0.330604] [G loss: -0.168288]
[Epoch 32/200] [Batch 345/938] [D loss: -0.304245] [G loss: -0.140743]
[Epoch

[Epoch 32/200] [Batch 865/938] [D loss: -0.274437] [G loss: -0.459975]
[Epoch 32/200] [Batch 870/938] [D loss: -0.287245] [G loss: -0.396914]
[Epoch 32/200] [Batch 875/938] [D loss: -0.314524] [G loss: -0.311560]
[Epoch 32/200] [Batch 880/938] [D loss: -0.313321] [G loss: -0.327020]
[Epoch 32/200] [Batch 885/938] [D loss: -0.281396] [G loss: -0.383904]
[Epoch 32/200] [Batch 890/938] [D loss: -0.306203] [G loss: -0.396902]
[Epoch 32/200] [Batch 895/938] [D loss: -0.292427] [G loss: -0.370858]
[Epoch 32/200] [Batch 900/938] [D loss: -0.308146] [G loss: -0.418628]
[Epoch 32/200] [Batch 905/938] [D loss: -0.287355] [G loss: -0.359909]
[Epoch 32/200] [Batch 910/938] [D loss: -0.288309] [G loss: -0.407416]
[Epoch 32/200] [Batch 915/938] [D loss: -0.322705] [G loss: -0.376226]
[Epoch 32/200] [Batch 920/938] [D loss: -0.297048] [G loss: -0.427291]
[Epoch 32/200] [Batch 925/938] [D loss: -0.312512] [G loss: -0.383783]
[Epoch 32/200] [Batch 930/938] [D loss: -0.274046] [G loss: -0.460163]
[Epoch

KeyboardInterrupt: 

In [3]:
WGAN.sample_path

NameError: name 'WGAN' is not defined